# 회원탈퇴 예측
- 클러스터링 행동 분석은 사용 방법에 따라 많은 가능성이 있는 기술이다. 
- 행동패턴을 분석 할 수 있으면 어떤 고객이 탈퇴할지와 같은 예측도 가능하다. 
- 탈퇴 방지를 하기위해 정책을 준비하는 것도 가능. 

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd 

In [5]:
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [7]:
year_months = list(uselog_months['연월'].unique())
uselog = pd.DataFrame()

for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months['연월'] == year_months[i], :]
    tmp.rename(columns = {'count':'count_0'}, inplace = True)
    tmp_before = uselog_months.loc[uselog_months['연월'] == year_months[i-1], :]
    del tmp_before['연월']
    tmp_before.rename(columns = {'count':'count_1'}, inplace = True)
    tmp = pd.merge(tmp, tmp_before, on = 'customer_id', how = 'left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)

In [8]:
uselog.head()

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### 탈퇴 전월의 탈퇴 고객 데이터를 작성
- 탈퇴한 월이 아닌 탈퇴 전월의 데이터를 작성한다. 
- 탈퇴를 예측하는 목적은 탈퇴방지를 위한 것.
- 해당 월에 탈퇴 신청하고 다음달 말일에 탈퇴 처리를 함
- 탈퇴 전월로부터 탈퇴 신청 확률을 예측해보자~

In [9]:
from dateutil.relativedelta import relativedelta

In [10]:
exit_customer = customer.loc[customer['is_deleted'] == 1]
exit_customer['exit_date'] = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])

In [11]:
for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(month=1)
    

In [12]:
exit_customer['연월'] = exit_customer['exit_date'].dt.strftime('%Y%m')
exit_customer.head(3)

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,연월
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,0_종일,10500,2_일반,3.0,3.0,3,3,0,2018-04-30,23,2018-01-30 00:00:00,201801
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,1_주간,7500,2_일반,3.0,3.0,3,3,0,2018-04-30,23,2018-01-30 00:00:00,201801
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,0_종일,10500,2_일반,1.0,1.0,1,1,0,2018-04-30,23,2018-01-30 00:00:00,201801


In [16]:
uselog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33851 entries, 0 to 33850
Data columns (total 4 columns):
연월             33851 non-null int64
customer_id    33851 non-null object
count_0        33851 non-null int64
count_1        32650 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.0+ MB


In [17]:
uselog['연월'] = uselog['연월'].astype(str)

In [18]:
exit_customer = pd.merge(uselog, exit_customer, on = ['customer_id', '연월'], how = 'left')

In [19]:
exit_customer.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
len(exit_customer)

33851

In [21]:
exit_customer.isna().sum()

연월                       0
customer_id              0
count_0                  0
count_1               1201
name                 33465
class                33465
gender               33465
start_date           33465
end_date             33465
campaign_id          33465
is_deleted           33465
class_name           33465
price                33465
campaign_name        33465
mean                 33465
median               33465
max                  33465
min                  33465
routine_flg          33465
calc_date            33465
membership_period    33465
exit_date            33465
dtype: int64

In [22]:
exit_customer = exit_customer.dropna(subset= ['name'])
print(len(exit_customer))
print(len(exit_customer['customer_id'].unique()))

386
386


### 지속회원의 데이터 작성 
- 샘플 수를 조정, 모든 기간의 회원 데이터를 사용하지 않고, 하나의 기간만 사용

In [24]:
conti_customer = customer.loc[customer['is_deleted'] == 0, :]
conti_uselog = pd.merge(uselog, conti_customer, on = ['customer_id'], how = 'left')

In [25]:
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset = ['name'])
print(len(conti_uselog))

33851
27422


In [26]:
conti_uselog = conti_uselog.sample(frac = 1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset= 'customer_id')
print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201812,GD763218,6,4.0,XX,C03,M,2015-10-01,NaN,CA1,...,2_야간,6000.0,2_일반,4.166667,4.0,6.0,3.0,1.0,2019-04-30,42.0
1,201901,AS466656,3,4.0,XXXXX,C01,F,2016-11-01,NaN,CA1,...,0_종일,10500.0,2_일반,4.666667,4.5,8.0,3.0,1.0,2019-04-30,29.0
2,201811,OA736386,3,5.0,XXXXX,C02,M,2016-06-01,NaN,CA1,...,1_주간,7500.0,2_일반,5.166667,5.0,8.0,3.0,1.0,2019-04-30,34.0
3,201811,HD995428,4,8.0,XXX,C02,F,2015-10-01,NaN,CA1,...,1_주간,7500.0,2_일반,3.916667,4.0,8.0,1.0,1.0,2019-04-30,42.0
4,201808,PL342449,7,3.0,XX,C02,F,2015-06-01,NaN,CA1,...,1_주간,7500.0,2_일반,4.166667,4.0,7.0,1.0,1.0,2019-04-30,46.0


In [ ]:
predict_data = pd.concat([conti_uselog, exit_customer])